# Graph clustering

In [ ]:
from sklearn.decomposition import PCA
from node2vec import Node2Vec as n2v
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import pandas as pd
import preprocessing.Preprocessing as pp
import classes.transportnetwork as tn
from visualisation.visualisation import *

import numpy as np
from sklearn.manifold import TSNE

In [ ]:
G = pp.create_network_from_trailway('../../../data/Railway Data_JL.xlsx')
G2 = pp.create_network_from_GTFS('../../../data/gtfs')

In [ ]:
TN = tn.TransportNetwork(G, pos_argument=["lon", "lat"], edges_weight_argument='train_max_speed')
#TN = tn.TransportNetwork(G2, pos_argument=['lon', 'lat'])

## Based on structural similirarity

### Structure role similarity

#### Node2vec p=1, q=2 + ML clustering

##### Node2vec p=1, q=2

In [66]:
WINDOW = 1 # Node2Vec fit window
MIN_COUNT = 1 # Node2Vec min. count
BATCH_WORDS = 4 # Node2Vec batch words

g_emb_struct = n2v(
    TN.multidigraph, # a graph g, where all nodes must be integers or strings
    dimensions=64, # embedding dimensions (default: 128)
    #walk_length=4, # number of nodes in each walk (default: 80)
    #num_walks=100, # number of walks per node (default: 10)
    #weight_key=None, # key in edge data for weight (default: None)
    workers=1, # number of workers (default: 1)
    p=0.25, # the probability for a random walk getting back to the prebious node (default: 1)
    q=4, # the probability that a random walk can pass through a previously unseen part of the graph (default: 1)
)

mdl_struct = g_emb_struct.fit(
    vector_size = 64,
    window=WINDOW,
    min_count=MIN_COUNT,
    batch_words=BATCH_WORDS
)

Computing transition probabilities:   0%|          | 0/2719 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 10/10 [00:28<00:00,  2.88s/it]


In [67]:
emb_df = (
    pd.DataFrame(
        [mdl_struct.wv.get_vector(str(n)) for n in TN.graph.nodes()],
        index = TN.graph.nodes
    )
)
plot_tsne_embedding(emb_df)

##### Spectral clustering

In [68]:
from sklearn.cluster import SpectralClustering

X = emb_df.values

clustering = SpectralClustering(
    n_clusters=4,
    assign_labels='discretize',
    random_state=0
).fit(X)

comm_dct = dict(zip(emb_df.index, clustering.labels_))
comm_dct = {k: v + 1 for k, v in comm_dct.items()}

map_weighted_network(TN, custom_node_weigth=comm_dct, edge_weigth=False, scale=2, node_size=5, discrete_color=True)

plot_tsne_embedding(emb_df, node_cluster=comm_dct)

##### K-means clustering

In [69]:
from sklearn.cluster import KMeans

X = emb_df.values

kmeans = KMeans(
    n_clusters= 4,
    random_state=0
).fit(X)

comm_dct = dict(zip(emb_df.index, kmeans.labels_))
comm_dct = {k: v + 1 for k, v in comm_dct.items()}

map_weighted_network(TN, custom_node_weigth=comm_dct, edge_weigth=False, scale=2, node_size=5, discrete_color=True)

plot_tsne_embedding(emb_df, node_cluster=comm_dct)

/home/anthony/Desktop/Group Project/XYZnetwork_lib/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



##### DBSCAN clustering

In [ ]:
from sklearn.cluster import DBSCAN

X = emb_df.values

dbscan = DBSCAN(
    eps=3.5,
    min_samples=10
).fit(X)

comm_dct = dict(zip(emb_df.index, dbscan.labels_))
comm_dct = {k: v + 1 for k, v in comm_dct.items()}

map_weighted_network(TN, custom_node_weigth=comm_dct, edge_weigth=False, scale=100, node_size=5, discrete_color=True)

plot_tsne_embedding(emb_df, node_cluster=dbscan.labels_)

##### Agglomerative clustering

In [ ]:
from sklearn.cluster import AgglomerativeClustering

X = emb_df.values

agg_clustering = AgglomerativeClustering(
    n_clusters=10,
    affinity='euclidean',
    linkage='ward'
).fit(X)

comm_dct = dict(zip(emb_df.index, agg_clustering.labels_))
comm_dct = {k: v + 1 for k, v in comm_dct.items()}

map_weighted_network(TN, custom_node_weigth=comm_dct, edge_weigth=False, scale=2, node_size=5, discrete_color=True)

plot_tsne_embedding(emb_df, node_cluster=agg_clustering.labels_)

#### Struct2vec + ML clustering

#### GraphWave

### Community similarity

#### Modularity algorithm

##### Louvain

In [ ]:
from community import community_louvain
comms = community_louvain.best_partition(TN.graph)
comms
map_weighted_network(TN, custom_node_weigth=comms, edge_weigth=False, scale=2, node_size=5, discrete_color=True)

##### Greedy modularity

In [ ]:
communities = nx.community.greedy_modularity_communities(TN.graph)

# Create a dictionnaire with the communities
comms_dct = {}
for i, comm in enumerate(communities):
    for node in comm:
        comms_dct[node] = i

map_weighted_network(TN, custom_node_weigth=comms_dct, edge_weigth=False, scale=2, node_size=5, discrete_color=True)

##### Label propagation

In [ ]:
communities = nx.community.label_propagation_communities(TN.graph)

# Create a dictionnaire with the communities
comms_dct = {}
for i, comm in enumerate(communities):
    for node in comm:
        comms_dct[node] = i

map_weighted_network(TN, custom_node_weigth=comms_dct, edge_weigth=False, scale=2, node_size=5, discrete_color=True)


#### Factorization embedding

##### Laplacian eigenmaps

In [ ]:
# Create the adjacency matrix
A = nx.to_numpy_array(TN.multigraph)

# Compute the degree matrix
D = np.diag(np.sum(A, axis=1))

# Compute the Laplacian matrix
L = D - A

# Compute the eigenvectors and eigenvalues of L
eigenvals, eigenvecs = np.linalg.eig(L)

# Sort the eigenvectors by their corresponding eigenvalues
idx = eigenvals.argsort()
eigenvals = eigenvals[idx]
eigenvecs = eigenvecs[:,idx]

# Select the k eigenvectors corresponding to the k smallest eigenvalues
k = 3
X = eigenvecs[:,:k]

# Normalize the rows of X
X_norm = np.linalg.norm(X, axis=1)
X_norm[X_norm==0] = 1
X = X / X_norm[:,None]

##### + Spectral clustering

In [ ]:
from sklearn.cluster import SpectralClustering

# Perform spectral clustering on the embedded data
clustering = SpectralClustering(n_clusters=30, assign_labels='discretize', random_state=0).fit(X)

comm_dct2 = dict(zip(emb_df.index, clustering.labels_))
comm_dct2 = {k: v + 1 for k, v in comm_dct2.items()}

map_weighted_network(TN, custom_node_weigth=comm_dct2, edge_weigth=False, scale=2, node_size=5)

##### + K-means clustering

In [ ]:
# Perform k-means clustering on the embedded data
kmeans = KMeans(n_clusters=30, random_state=0).fit(X)

comm_dct2 = dict(zip(emb_df.index, kmeans.labels_))
comm_dct2 = {k: v + 1 for k, v in comm_dct2.items()}

map_weighted_network(TN, custom_node_weigth=comm_dct2, edge_weigth=False, scale=2, node_size=5, discrete_color=True)

##### Adjenct matrix embedding

In [ ]:
# convert the graph to a matrix
A = nx.to_numpy_array(TN.multidigraph)

##### + Spectral clustering

In [ ]:
from sklearn.cluster import SpectralClustering

# perform spectral clustering
sc = SpectralClustering(n_clusters=30, affinity='precomputed', assign_labels='kmeans')
sc.fit(A)

comm_dct2 = dict(zip(emb_df.index, sc.labels_))
comm_dct2 = {k: v + 1 for k, v in comm_dct2.items()}

map_weighted_network(TN, custom_node_weigth=comm_dct2, edge_weigth=False, scale=2, node_size=5, discrete_color=True)

##### + K-means clustering

In [ ]:
# perform k-means clustering
kmeans = KMeans(n_clusters=12, random_state=0).fit(A)

comm_dct2 = dict(zip(emb_df.index, kmeans.labels_))
comm_dct2 = {k: v + 1 for k, v in comm_dct2.items()}

map_weighted_network(TN, custom_node_weigth=comm_dct2, edge_weigth=False, scale=2, node_size=5, discrete_color=True)

#### Node2vec p=1, q=0.5 + ML clustering

In [ ]:
WINDOW = 10 # Node2Vec fit window
MIN_COUNT = 1 # Node2Vec min. count
BATCH_WORDS = 4 # Node2Vec batch words

g_emb_struct = n2v(
    TN.multidigraph, # a graph g, where all nodes must be integers or strings
    dimensions=64, # embedding dimensions (default: 128)
    # walk_length=16, # number of nodes in each walk (default: 80)
    #num_walks=100, # number of walks per node (default: 10)
    #weight_key=None, # key in edge data for weight (default: None)
    workers=1, # number of workers (default: 1)
    p=1, # the probability for a random walk getting back to the prebious node (default: 1)
    q=0.5, # the probability that a random walk can pass through a previously unseen part of the graph (default: 1)
)

mdl_struct = g_emb_struct.fit(
    vector_size = 64,
    window=WINDOW,
    min_count=MIN_COUNT,
    batch_words=BATCH_WORDS
)

emb_df = (
    pd.DataFrame(
        [mdl_struct.wv.get_vector(str(n)) for n in TN.graph.nodes()],
        index = TN.graph.nodes
    )
)

##### + Spectral clustering

In [ ]:
from sklearn.cluster import SpectralClustering

X = emb_df.values

clustering = SpectralClustering(
    n_clusters=30,
    assign_labels='discretize',
    random_state=0
).fit(X)

comm_dct = dict(zip(emb_df.index, clustering.labels_))
comm_dct = {k: v + 1 for k, v in comm_dct.items()}


map_weighted_network(TN, custom_node_weigth=comm_dct, edge_weigth=False, scale=2, node_size=5)
plot_tsne_embedding(emb_df, node_cluster=comm_dct)

##### + K-means clustering

In [ ]:
from sklearn.cluster import KMeans

X = emb_df.values

kmeans = KMeans(
    n_clusters=5,
    random_state=0
).fit(X)

comm_dct = dict(zip(emb_df.index, kmeans.labels_))
comm_dct = {k: v + 1 for k, v in comm_dct.items()}

map_weighted_network(TN, custom_node_weigth=comm_dct, edge_weigth=False, scale=2, node_size=5, discrete_color=True)
plot_tsne_embedding(emb_df, node_cluster=comm_dct)

## Based on feature similarity

In [ ]:
# Create a new transport network
G = pp.create_network_from_trailway("../../../data/Railway Data_JL.xlsx")
TN = tn.TransportNetwork(G, pos_argument=['lon', 'lat'], time_arguments=["dep_time", "arr_time"], nodes_weight_argument="lat", edges_weight_argument="train_max_speed", distance_argument="distance")
# G2 = pp.create_network_from_GTFS("../../../data/gtfs")
# TN = tn.TransportNetwork(G2, pos_argument=['lon', 'lat'])
# G3 = pp.create_network_from_edges("../../../data/road-euroroad.edges")
# TN = tn.TransportNetwork(G3)

# graph = TN.get_higher_complexity()
#
# # Compute the detour for each edge and add it to their attributes
# euclidian_distance, real_distance, detour = compute_distances_analysis(TN, data=True)
#
# # Normalize the euclidian distance and minus 1
# # weigth = {k: v / max(euclidian_distance.values()) for k, v in euclidian_distance.items()}
# weigth = {k: max(euclidian_distance.values()) - v for k, v in euclidian_distance.items()}
#
# # Create random weights
# weigth = {}
# for edge in list(graph.edges):
#     weigth[edge] = random.random()

# # Replace the 0 values by the minimum value
# print(euclidian_distance)

# # Extract values and reshape to have a single feature
# values = list(euclidian_distance.values())
# values = [[val] for val in values]
#
# # Create MinMaxScaler instance with feature_range=(0.01, 1) and fit to values
# scaler = MinMaxScaler(feature_range=(0, 1))
# scaler.fit(values)
#
# # Transform values using MinMaxScaler
# transformed_values = scaler.transform(values)
#
# # Update dictionary with transformed values
# for i, key in enumerate(euclidian_distance.keys()):
#     euclidian_distance[key] = transformed_values[i][0]
#     # Perform 1 - value to have a value between 0 and 1
#     euclidian_distance[key] = 1 - euclidian_distance[key]

# Add the euclidian distance to the graph
# nx.set_edge_attributes(graph, weigth, "weight")
#
# for edge in graph.edges(data=True):
#     print(edge)


from sklearn.cluster import SpectralClustering
from sklearn.cluster import KMeans

WINDOW = 10 # Node2Vec fit window
MIN_COUNT = 1 # Node2Vec min. count
BATCH_WORDS = 4 # Node2Vec batch words

g_emb_struct = n2v(
    TN.graph, # a graph g, where all nodes must be integers or strings
    dimensions=64, # embedding dimensions (default: 128)
    # walk_length=16, # number of nodes in each walk (default: 80)
    #num_walks=100, # number of walks per node (default: 10)
    # weight_key="weight", # key in edge data for weight (default: None)
    workers=1, # number of workers (default: 1)
    p=1, # the probability for a random walk getting back to the prebious node (default: 1)
    q=0.5, # the probability that a random walk can pass through a previously unseen part of the graph (default: 1)
)

mdl_struct = g_emb_struct.fit(
    vector_size = 64,
    window=WINDOW,
    min_count=MIN_COUNT,
    batch_words=BATCH_WORDS
)

emb_df = (
    pd.DataFrame(
        [mdl_struct.wv.get_vector(str(n)) for n in TN.graph.nodes()],
        index = TN.graph.nodes
    )
)

X = emb_df.values

clustering = SpectralClustering(
    n_clusters=30,
    assign_labels='discretize',
    random_state=21,
).fit(X)

comm_dct = dict(zip(emb_df.index, clustering.labels_))
comm_dct = {k: v + 1 for k, v in comm_dct.items()}


plot_tsne_embedding(emb_df, node_cluster=comm_dct)
plot_tsne_embedding(emb_df)
map_weighted_network(TN, custom_node_weigth=comm_dct, edge_weigth=False, scale=2, node_size=5, discrete_color=True)

In [ ]:
import lib.graphwave
from lib.graphwave.graphwave import *

In [ ]:
chi, heat_print, taus = graphwave_alg(TN.graph, np.linspace(0,100,25), taus='auto', verbose=True)

In [ ]:
# create a dataframe with the node id as index and the chi values as columns
chi_df = pd.DataFrame(chi, index=TN.graph.nodes)
chi_df

In [ ]:
plot_tsne_embedding(chi_df)

In [ ]:
from sklearn.decomposition import PCA
km = KMeans(n_clusters=4)
km.fit(chi)
labels_pred=km.labels_

In [ ]:
comm_dct = dict(zip(chi_df.index, km.labels_))
comm_dct = {k: v + 1 for k, v in comm_dct.items()}

map_weighted_network(TN, custom_node_weigth=comm_dct, edge_weigth=False, scale=2, node_size=5, discrete_color=True)
plot_tsne_embedding(chi_df, node_cluster=comm_dct)